In [ ]:
#!/usr/bin/env python
import ROOT, sys, os
from ROOT import std

from larlite import larlite
from larcv import larcv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.path as path
colormap=['blue','red','magenta','green','orange','yellow','pink']
nc=len(colormap)

In [ ]:
iom=larcv.IOManager(larcv.IOManager.kWRITE)        
iom.reset()
iom.set_verbosity(0)                                                                           
#iom.add_in_file("poop.root")
iom.set_out_file("trashme_aho.root") 
iom.initialize()

In [ ]:
shape_ones=np.ones((864, 756))
shape_zeros=np.zeros((864, 756))

In [ ]:
#make the letter K
shape1=np.array([[50,100],[50,350],[75,350],
                 [75,200],[175,300],[150,250],
                 [75,175],[150,100],[125,100],
                 [75,150],[75,100],[50,100]])
#shape1=np.array([[50,100],[50,300],[150,300],
#                 [70,250],[150,100],[50,100]])
shape2=shape1[:,::-1].copy()+100
shape3=shape1[:,::-1].copy()+300
shape4=shape1[:,::-1].copy()+450
#plt.plot(shape1[:,0],shape1[:,1],'o-')
#plt.plot(shape2[:,0],shape2[:,1],'o-')
#plt.plot(shape3[:,0],shape3[:,1],'o-')
#plt.plot(shape4[:,0],shape4[:,1],'o-')
#ax=plt.gca()
#ax.set_xlim(0,800)
#ax.set_ylim(0,800)

In [ ]:
path1=path.Path(shape1)
path2=path.Path(shape2)
path3=path.Path(shape3)
path4=path.Path(shape4)

In [ ]:
nz=np.fliplr(np.vstack(np.nonzero(shape_ones)).T)

inside=nz[path1.contains_points(nz,radius=1)]
shape_zeros[inside[:,0],inside[:,1]]=200

inside=nz[path2.contains_points(nz,radius=1)]
shape_zeros[inside[:,0],inside[:,1]]=200

inside=nz[path3.contains_points(nz,radius=1)]
shape_zeros[inside[:,0],inside[:,1]]=200

inside=nz[path4.contains_points(nz,radius=1)]
shape_zeros[inside[:,0],inside[:,1]]=200

shape_img=shape_zeros.copy().T

In [ ]:
#plt.imshow(shape_img,cmap='Greys',interpolation='none')

In [ ]:
ev_img=iom.get_data(larcv.kProductImage2D,"tpc")
for i in xrange(3):
    im2d=larcv.as_image2d(shape_img.astype(np.float32),
                     larcv.ImageMeta(shape_img.shape[1],shape_img.shape[0],shape_img.shape[1],shape_img.shape[0],0,0,i))
    ev_img.Append(im2d)


In [ ]:
larbysimg=larcv.LArbysImage()
cfg=larcv.CreatePSetFromFile("unit.fcl","LArbysImage")
larbysimg.configure(cfg)
larbysimg.initialize()
larbysimg.process(iom)
#larbysimg.process(iom)

In [ ]:
print larbysimg.Manager().GetClusterAlg(0)
print larbysimg.Manager().GetClusterAlg(1)
print larbysimg.Manager().GetClusterAlg(2)

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
plt.imshow(shape_img[:,::-1],cmap='Greys',interpolation='none')
mgr=larbysimg.Manager()
dm=mgr.DataManager()
data=dm.Data(0)
for x in xrange(data._mip_ctors_v_v[0].size()):
    pts=[ [pt.x,pt.y] for pt in np.array(data._mip_ctors_v_v[0][x]) ]
    pts=np.array(pts)
    plt.plot(pts[:,0],pts[:,1],lw=2)
ax=plt.gca()
ax.set_title("MIP Contours")

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
plt.imshow(shape_img[:,::-1],cmap='Greys',interpolation='none')

cv=larbysimg.Manager().GetClusterAlg(1)
mgr=larbysimg.Manager()
dm=mgr.DataManager()
dc_data=dm.Data(1)

dc_atomics=dc_data._atomic_ctor_v_v[0]
for atomic in dc_atomics:
    pts=[ [pt.x,pt.y] for pt in np.array(atomic) ]
    pts=np.array(pts)
    plt.plot(pts[:,0],pts[:,1],lw=2)
    
ax=plt.gca()
ax.set_xlim(0,900)
ax.set_ylim(900,0)
ax.set_title("Split Contours")
plt.show()

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
plt.imshow(shape_img[:,::-1],cmap='Greys',interpolation='none')

cv=larbysimg.Manager().GetClusterAlg(2)
mgr=larbysimg.Manager()
dm=mgr.DataManager()
cv_data=dm.Data(2)

cv_ctor_lines_v=cv_data._ctor_lines_v_v_v[0]
cv_inter_pts_v=cv_data._ipoints_v_v_v[0]
ix=-1

for cv_ctor_lines in cv_ctor_lines_v:
    print "==",cv_ctor_lines.size(),"=="
    
    ix+=1
    color=colormap[ix]
    
    for line in cv_ctor_lines:
        plt.plot([0,900],[line.y(0),line.y(900)],alpha=0.7,color=color)

    for inter_pt in cv_inter_pts_v[ix]:
        plt.plot(inter_pt.x,inter_pt.y,'o',markersize=10,color=color,alpha=0.9)   
        
    print "===="
ax=plt.gca()
#ax.set_xlim(0,900)
#ax.set_ylim(900,0)
ax.set_xlim(200,250)
ax.set_ylim(520,500)
ax.set_title("PCA on Contour, Intersection PTS")
plt.show()

In [ ]:
import matplotlib.patches as patches

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
plt.imshow(shape_img[:,::-1],cmap='Greys',interpolation='none')

cv=larbysimg.Manager().GetClusterAlg(3)
mgr=larbysimg.Manager()
dm=mgr.DataManager()
cv_data=dm.Data(3)

circledata_v=cv_data._circledata_v_v[0]
for circle in circledata_v:
    c=patches.Circle((circle.center.x,circle.center.y),
                  circle.radius,ec='cyan',fc='none',lw=2)
    ax.add_patch(c)

ax=plt.gca()
ax.set_xlim(0,900)
ax.set_ylim(900,0)
plt.show()